In [5]:
#!pip install python-whois
#!pip install requests
#!pip install beautifulsoup4

In [6]:
import whois
import requests
import time

from bs4 import BeautifulSoup

In [7]:
#Se recuperan la información de los principales Estadios españoles de la wikipedia

urlEstadios = "https://es.wikipedia.org/wiki/Anexo:Estadios_de_f%C3%BAtbol_de_Espa%C3%B1a"

pageEstadios = requests.get(urlEstadios)
soupEs = BeautifulSoup(pageEstadios.content)

tables = soupEs.find_all('table')

tbody = tables[0].find('tbody')
trs = tbody.find_all('tr')

del trs[0]

ciudades = []
#estadios = []
equipos = []

for tr in trs:
    tds = tr.find_all('td')
       
    if tds[0].string.isdigit():
        del tds[0]
    
    if(len(tds) >= 7):
        ciudades.append(tds[1].find('a').string)
    elif (len(tds) == 6):
        ciudades.append(ciudades[-1])

    #estadios.append(tds[0].find('a').string)
    try:
        equipos.append(tds[-2].find('a').string)
    except AttributeError as e:
        equipos.append(tds[-2].string)
        
tbody = tables[1].find('tbody')
trs = tbody.find_all('tr')

del trs[0]

i = 0

for tr in trs:
    
    if i == 90:
        break

    tds = tr.find_all('td')
       
    if tds[0].string.isdigit():
        del tds[0]
    
    if(len(tds) >= 6):
        #print(tds[1].find('a').string)
        ciudades.append(tds[1].find('a').string)
    elif (len(tds) == 5):
        #print(ciudades2[-1])
        ciudades.append(ciudades[-1])

    #estadios.append(tds[0].find('a').string)
    equipos.append(tds[-1].find('a').string)
    i = i + 1   

dic = {}
for ciudad, equipo in zip(ciudades, equipos):
    
    equipo = equipo.replace("C. F.", "").strip()
    equipo = equipo.replace("F. C.", "").strip()
    equipo = equipo.replace("C. D.", "").strip()
    equipo = equipo.replace("U. D.", "").strip()
    equipo = equipo.replace("FC", "").strip()
    equipo = equipo.replace("RCD", "").strip()
    equipo = equipo.replace("R. C.", "").strip()
    equipo = equipo.replace("R.", "").strip()    
    equipo = equipo.replace("CD", "").strip()  
    equipo = equipo.replace("C. A.", "").strip() 
    equipo = equipo.replace("U.D.", "").strip() 
    equipo = equipo.replace("Club ", "").strip() 
    equipo = equipo.replace("Real ", "").strip() 
    
    
    dic[equipo] = ciudad

    
#Se añaden lo que estan con error
dic['Real Madrid'] = 'Madrid'
dic['Alavés'] = 'Vitoria'
dic['Real Sociedad'] = 'San Sebastián'


print(dic)

{'Barcelona': 'Barcelona', 'Madrid': 'Madrid', 'Atlético de Madrid': 'Madrid', 'Betis Balompié': 'Sevilla', 'En mantenimiento': 'Sevilla', 'Desocupado': 'Barcelona', 'Valencia': 'Valencia', 'Athletic Club': 'Bilbao', 'Sevilla': 'Sevilla', 'Espanyol': 'Cornellá de Llobregat', 'Sociedad de Fútbol': 'San Sebastián', 'Elche': 'Elche', 'Zaragoza': 'Zaragoza', 'Deportivo de La Coruña': 'La Coruña', 'Las Palmas': 'Las Palmas de Gran Canaria', 'Murcia': 'Murcia', 'Oviedo': 'Oviedo', 'Málaga': 'Málaga', 'Sporting de Gijón': 'Gijón', 'Hércules': 'Alicante', 'Celta de Vigo': 'Vigo', 'Valladolid': 'Valladolid', 'Levante': 'Valencia', 'Osasuna': 'Pamplona', 'Villarreal': 'Villarreal', 'Mallorca': 'Palma', 'Tenerife': 'Santa Cruz de Tenerife', 'Racing de Santander': 'Santander', 'Recreativo de Huelva': 'Huelva', 'Córdoba': 'Córdoba', 'Xerez Deportivo': 'Jerez de la Frontera', 'Cádiz': 'Cádiz', 'Deportivo Alavés': 'Vitoria', 'Granada': 'Granada', 'Salamanca CF UDS': 'Villares de la Reina', 'Albacete 

In [8]:
class linea(object):
    
    def __init__(self, equipoAnf, equipoVis, resultadoAnf, resultadoVis, fecha, estadio):
        self.equipoAnf = equipoAnf
        self.equipoVis = equipoVis 
        self.resultadoAnf = resultadoAnf 
        self.resultadoVis = resultadoVis
        self.fecha = fecha
        self.estadio = estadio
        
        try:
            self.ciudad = dic[equipoAnf]
        except KeyError:
            self.ciudad = "n/d"
        
        
    @property
    def equipoAnf(self):
        return self._equipoAnf
    
    @equipoAnf.setter
    def equipoAnf(self, value):
        self._equipoAnf = value
    
    @property
    def equipoVis(self):
        return self._equipoVis
    
    @equipoVis.setter
    def equipoVis(self, value):
        self._equipoVis = value
    
    @property
    def resultadoAnf(self):
        return self._resultadoAnf
    
    @resultadoAnf.setter
    def resultadoAnf(self, value):
        self._resultadoAnf = value    

    @property
    def resultadoVis(self):
        return self._resultadoVis

    @resultadoVis.setter
    def resultadoVis(self, value):
        self._resultadoVis = value    

    @property
    def fecha(self):
        return self._fecha

    @fecha.setter
    def fecha(self, value):
        self._fecha = value    
        
    @property
    def estadio(self):
        return self._estadio

    @estadio.setter
    def estadio(self, value):
        self._estadio = value      
        
    @property
    def ciudad(self):
        return self._ciudad

    @ciudad.setter
    def ciudad(self, value):
        self._ciudad = value    
    


In [9]:
print(whois.whois('https://www.bdfutbol.com/'))

{
  "domain_name": [
    "BDFUTBOL.COM",
    "bdfutbol.com"
  ],
  "registrar": "1&1 IONOS SE",
  "whois_server": "whois.ionos.com",
  "referral_url": null,
  "updated_date": [
    "2020-01-22 08:30:25",
    "2017-02-12 10:43:58"
  ],
  "creation_date": "2008-01-21 16:08:14",
  "expiration_date": "2021-01-21 16:08:14",
  "name_servers": [
    "NS1026.UI-DNS.BIZ",
    "NS1047.UI-DNS.DE",
    "NS1054.UI-DNS.ORG",
    "NS1060.UI-DNS.COM"
  ],
  "status": [
    "clientTransferProhibited https://icann.org/epp#clientTransferProhibited",
    "clientTransferProhibited https://www.icann.org/epp#clientTransferProhibited",
    "autoRenewPeriod https://www.icann.org/epp#autoRenewPeriod"
  ],
  "emails": [
    "abuse@ionos.com",
    "dataprivacyprotected@1und1.de"
  ],
  "dnssec": [
    "unsigned",
    "Unsigned"
  ],
  "name": "REDACTED FOR PRIVACY",
  "org": null,
  "address": "REDACTED FOR PRIVACY",
  "city": "REDACTED FOR PRIVACY",
  "state": "GI",
  "zipcode": "REDACTED FOR PRIVACY",
  "countr

In [10]:
main = 'https://www.bdfutbol.com/es/'
tt    = 't/t'
html = '.html'
guion = '-'

urlMainList = []


for i in range(0, -1, -1):
#Se recorre las Urls para confeccionar la lista de Url's
#for i in range(20, -1, -1):    
    anio1 = str(2000 + i)
    
    if i < 9:
        anio2 = "0" + str(i + 1)
    else:
        anio2 = str(i + 1)
    

    url = main + tt + anio1 + guion + str(anio2) + html
    
    urlMainList.append(url)



In [11]:
print(urlMainList)
urlMainList[0] = 'https://www.bdfutbol.com/es/t/t2001-02.html'

['https://www.bdfutbol.com/es/t/t2000-01.html']


In [12]:
list_lineas=[]

#Se recorren todas las URL's para recuperar los datos de todas temporadas desde el año 2000 al 2020
for url in urlMainList:
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content)
    jornadas = soup.find(id="jornada_classi")
    links = jornadas.find_all('a')
    href = []

    #Se recorre la lista de etiquetas a para coger unicamente la href
    for link in links:
        href.append(link.get('href'))
    
    #Se eliminan duplicados
    href_set = set(href)
    
    href_set = list(map(lambda x: x.replace("../", main), href_set))

    for link in href_set[:70]:
    
        t0 = time.time()
    
        partido = requests.get(link)
        p_soup = BeautifulSoup(partido.content)
    
        # estimación del tiempo de respuesta en segundos
        response_delay = time.time() - t0
        # espera de 10x, con respecto al tiempo de respuesta
        time.sleep(10 * response_delay)
    
        Lista = p_soup.title.string.split("-")
        Equipo1 = Lista[0].strip()
        Equipo2 = Lista[1].split("(")[0].strip()
        Resul1 = Lista[1].split("(")[1].strip()
        Resul2 = Lista[2].split(")")[0].strip()
        Fecha = Lista[2].split(")")[1].strip()
        Estadio = p_soup.find_all("div", class_="font-weight-bold")[2].string.strip()
    
        list_lineas.append(linea(Equipo1, Equipo2, Resul1, Resul2, Fecha, Estadio))

In [ ]:
import csv
filename = 'Partidos.csv'

try:
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f)
        for linea in list_lineas:
            writer.writerow([linea.equipoAnf, linea.equipoVis, linea.resultadoAnf, linea.resultadoVis, linea.fecha, linea.estadio, linea.ciudad])
except BaseException as e:
    print('BaseException:', filename)
else:
    print('Data has been loaded successfully !')